In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

In [3]:
ubicacion_df = pd.read_excel("datos_cajero/DatosQF78.xlsx", sheet_name="Ubicación Cajeros")
ubicacion_df.head()

,Cajero,Coordenada en X,Coordenada en Y,Costo Fijo por Stock Out,Costo Variable por Stock Out,Duración de la recarga [Horas],Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo,Turno mañana,Turno tarde,Turno noche
0,Cajero 1,50,104,0.39,0.04,0.2,1,1,1,1,1,1,1,0,0,1
1,Cajero 2,24,24,0.37,0.02,0.2,0,1,1,1,1,1,0,1,1,1
2,Cajero 3,9,48,0.44,0.05,0.3,1,1,1,1,0,0,1,1,1,1
3,Cajero 4,79,73,0.30,0.05,0.2,1,1,0,1,1,0,1,1,1,1
4,Cajero 5,13,20,0.23,0.04,0.3,1,1,0,1,1,1,1,1,1,1


In [4]:
retiros_df = pd.read_excel("datos_cajero/DatosQF78.xlsx", sheet_name="Retiros")
retiros_df.head()

,Día,Cajero,Monto,Hora de la transacción
0,1,Cajero 196,2.70,0:0
1,1,Cajero 14,2.92,0:1
2,1,Cajero 143,5.69,0:3
3,1,Cajero 142,6.74,0:4
4,1,Cajero 214,2.40,0:6


---
# Formativo 1

# P1

In [5]:
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

def at_least_n_days(row, n):
    values = row.values.tolist()
    if sum(values) >= n:
        return 1
    else: 
        return 0


ubicacion_df['Reestablecido >=4 días'] = ubicacion_df[ubicacion_df.columns[6:13]].apply(lambda x: at_least_n_days(x, 4), axis=1)
len(ubicacion_df[ubicacion_df['Reestablecido >=4 días'] == 1])

212

# P2

In [6]:
def at_least_n_days_wknd_notnight(row, n):
    values = row.values.tolist()
    cond_1 = sum(values[:7]) >= n # Al menos 4 dias de la semana
    cond_2 = 1 in values[5:7] # Al menos un dia en fin de semana
    cond_3 = values[-1] == 0 # Sin turno de noche
    if cond_1 & cond_2 & cond_3:
        return 1
    else: 
        return 0
    
ubicacion_df['4days_wknd_notnight'] = ubicacion_df[ubicacion_df.columns[6:16]].apply(lambda x: at_least_n_days_wknd_notnight(x, 4), axis=1)
len(ubicacion_df[ubicacion_df['4days_wknd_notnight'] == 1])

53

# P3

In [7]:
def calculate_total_amount(cajero):
    grouped_by_atm = retiros_df.groupby(['Cajero', 'Día']).sum()
    amount = grouped_by_atm.loc[cajero]['Monto'].sum() 
    return amount

calculate_total_amount('Cajero 214')

2072.49

# P4

In [8]:
cajeros = ubicacion_df['Cajero'].unique().tolist()

rentables = 0
for caj in cajeros:
    if calculate_total_amount(caj) > 2200:
        rentables += 1

rentables

103

# P5

In [9]:
retiros_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98222 entries, 0 to 98221
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Día                     98222 non-null  int64  
 1   Cajero                  98222 non-null  object 
 2   Monto                   98222 non-null  float64
 3   Hora de la transacción  98222 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.0+ MB


In [10]:
def cashout_2am(row):
    hour_minute = row.split(':')
    hour = int(hour_minute[0])
    minute = int(hour_minute[1])
    if hour == 2 and minute >= 0 and minute <= 59:
        return 1
    else:
        return 0


retiros_df['2-2:59'] = retiros_df['Hora de la transacción'].apply(cashout_2am)
len(retiros_df[retiros_df['2-2:59'] == 1])

4120

---
# Formativo 2

In [15]:
def cashout_by_hour(row_hour, given_hour):
    hour_minute = row_hour.split(':')
    hour = int(hour_minute[0])
    minute = int(hour_minute[1])
    if hour == given_hour and minute >= 0 and minute <= 59:
        return 1
    else:
        return 0

In [19]:
hours = [i for i in range(24)]
for hour in hours:
    retiros_df[f'{hour}-{hour}:59'] = retiros_df.apply(lambda x: cashout_by_hour(x['Hora de la transacción'], hour), axis=1)

In [29]:
agrupado_por_hora = retiros_df[retiros_df.columns[4:]].sum().sort_values(ascending=False)
agrupado_por_hora

13-13:59    4153
17-17:59    4143
14-14:59    4138
23-23:59    4135
3-3:59      4130
20-20:59    4129
2-2:59      4120
9-9:59      4117
7-7:59      4108
6-6:59      4104
16-16:59    4103
15-15:59    4094
11-11:59    4094
10-10:59    4094
4-4:59      4084
12-12:59    4083
21-21:59    4080
5-5:59      4071
8-8:59      4063
18-18:59    4063
19-19:59    4048
22-22:59    4047
1-1:59      4031
0-0:59      3990
dtype: int64

In [34]:
hora_mayor = agrupado_por_hora.index[0]
hora_menor = agrupado_por_hora.index[-1]

In [35]:
hora_mayor

'13-13:59'

In [37]:
hora_menor

'0-0:59'